In [1]:
import os

import nivapy3 as nivapy
import pandas as pd
import teotil2 as teo
from sqlalchemy import text

In [2]:
eng = nivapy.da.connect()

Username:  ········
Password:  ········


Connection successful.


# Remove selected large wastewater sites from TEOTIL2

The Martini model already explicitly includes wastewater discharges from some of the largest sites around Oslofjord. To avoid double counting, we need to remove these sites from the TEOTIL results.

**This notebook uses the TEOTIL2 model and the old input database in RESA**, as TEOTIL3 is not ready yet.

## 1. Identify site codes for wastewater plants included in Martini

Phil has provided details of the sites already included in the Martini model (see e-mail received 29.11.2023 at 10.20). Phil's spreadsheet does not include site codes, so the first step is to identify the correct `anleggs_nr` for each site.

In [3]:
# Read data from Phil
xl_path = r"../data/Flux_Martini_Rivers_CVO.xlsx"
mar_df = pd.read_excel(xl_path, sheet_name="RA_2018")
mar_df.head()

,Navn,Kortnavn,Kommune,Treatment principle,Resipient,Martini river ID,Q_L_s,TN_ton_yr,TP_ton_yr,BOF_ton_yr,KOF_ton_yr,ANS Depth (m),Outlet depth (m),Overflow depth (m),Outlet latitude (estimated),Outlet longitude,Comment
0,Remmendalen RA,REM,Halden,Biological-post-chemical,Iddefjorden,6,75.0,114.13,0.970,31.4,207.680,NaN,10.0,NaN,NaN,NaN,Outlet location not resolved
1,Bakke,BAK,Halden,Chemical-biological,Iddefjorden,7,NaN,39.10,0.391,10.9,29.846,NaN,NaN,NaN,NaN,NaN,Outlet location probably not resolved
2,Øra RA,ØRA,Fredrikstad,Chemical,Øra/Glomma,8,NaN,476.90,4.943,709.3,1712.180,0.0,0.0,NaN,NaN,NaN,Outlet location probably not resolved
3,Alvein RA,ALV,Sarpsborg,?,Glomma,9,NaN,243.00,1.930,418.3,959.570,0.0,0.0,NaN,NaN,NaN,Released in river
4,Hestevold RA,HES,Råde,Chemical,Krokstadfjorden,12,NaN,21.90,0.220,29.3,67.200,NaN,40.0,25.0,59.315,10.785,No nearby MARTINI river


The code below attempts to match sites in the TEOTIL2 database to the names given in Phil's spreadsheet.

In [4]:
# Search TEOTIL2 database based on name
df_list = []
for idx, row in mar_df.iterrows():
    martini_name = row["Navn"]
    name = martini_name.rstrip(" RA")
    sql = text(
        "SELECT * FROM resa2.rid_punktkilder "
        "WHERE anlegg_navn LIKE :name "
        "AND type = 'RENSEANLEGG'"
    )
    match_df = pd.read_sql(sql, eng, params={"name": "%" + name + "%"})
    
    # If we get a single match, store it for later, otherwise print additional info
    # for further investigation
    if len(match_df) == 1:
        match_df["martini_name"] = martini_name
        df_list.append(match_df)
    elif len(match_df) > 1:
        print(f"{len(match_df)} matches found for '{name}'.")
        display(match_df)
    else:
        print(f"No matches found for '{name}'.")

match_df = pd.concat(df_list, axis="rows")
print(f"\nThe following {len(match_df)} RAs were successfully matched:")
match_df

2 matches found for 'Bakke'.


,anlegg_nr,anlegg_navn,etat,opprettet,nedlagt,type,kno,adresse,postnr,regine,lon_utl,lat_utl,lon_anl,lat_anl,status
0,0101AL02,Bakke,None,None,None,RENSEANLEGG,101,None,None,001.2220,11.44376,59.01952,None,None,None
1,0711AL03,Bakken RA,None,None,None,RENSEANLEGG,711,None,None,012.1,10.35402,59.68566,None,None,None


2 matches found for 'Øra'.


,anlegg_nr,anlegg_navn,etat,opprettet,nedlagt,type,kno,adresse,postnr,regine,lon_utl,lat_utl,lon_anl,lat_anl,status
0,0106AL00,Renseanlegg Øra,None,None,None,RENSEANLEGG,106,None,None,002.12,10.96781,59.18309,None,None,None
1,1724AL00,Øra,None,None,None,RENSEANLEGG,1724,None,None,129.42,11.22185,64.06548,None,None,None


No matches found for 'Alvein'.
2 matches found for 'Fuglevik'.


...

,anlegg_nr,anlegg_navn,etat,opprettet,nedlagt,type,kno,adresse,postnr,regine,lon_utl,lat_utl,lon_anl,lat_anl,status
0,0602AL06,Solumstrand Renseanle,None,None,None,RENSEANLEGG,602,None,None,012.1,10.26730,59.71091,None,None,None
1,0602AL48,Solumstrand Silstasjo,None,None,None,RENSEANLEGG,602,None,None,012.1,10.26667,59.71299,None,None,None


No matches found for 'Tønsberg'.
2 matches found for 'Lillevik'.


,anlegg_nr,anlegg_navn,etat,opprettet,nedlagt,type,kno,adresse,postnr,regine,lon_utl,lat_utl,lon_anl,lat_anl,status
0,0709AL01,Lillevik Renseanlegg,None,None,None,RENSEANLEGG,709,None,None,015.42,10.025880,59.016630,None,None,None
1,1717AL21,Lillevik,None,None,None,RENSEANLEGG,1717,None,None,125.42,10.899797,63.611427,None,None,None


No matches found for 'Knarrdalstrand'.

The following 17 RAs were successfully matched:


,anlegg_nr,anlegg_navn,etat,opprettet,nedlagt,type,kno,adresse,postnr,regine,lon_utl,lat_utl,lon_anl,lat_anl,status,martini_name
0,0101AL07,Remmendalen,None,None,None,RENSEANLEGG,101,None,None,001.31Z,11.36011,59.12080,None,None,None,Remmendalen RA
0,0135AL01,Hestevold renseanlegg,None,None,None,RENSEANLEGG,135,None,None,003.120,10.79192,59.31496,None,None,None,Hestevold RA
0,0104AL01,Kambo,None,None,None,RENSEANLEGG,104,None,None,003.20,10.68650,59.47445,None,None,None,Kambo RA
0,0215AL35,Frogn,None,None,None,RENSEANLEGG,215,None,None,004.3,10.64256,59.64207,None,None,None,Frogn RA
0,0301AL01,Bekkelaget,None,None,None,RENSEANLEGG,301,None,None,006.21,10.76701,59.88296,None,None,None,Bekkelaget RA
0,0626AL61,Linnes Renseanlegg,None,None,None,RENSEANLEGG,626,None,None,011.A0,10.28835,59.74977,None,None,None,Linnes RA
0,0602AL45,Muusøya Renseanlegg,None,None,None,RENSEANLEGG,602,None,None,012.A10,10.15668,59.75055,None,None,None,Muusøya RA
0,0702AL30,Holmestrand,None,None,None,RENSEANLEGG,702,None,None,013.2,10.32588,59.48517,None,None,None,Holmestrand RA
0,0701AL01,Falkensten,None,None,None,RENSEANLEGG,701,None,None,013.32,10.44839,59.43885,None,None,None,Falkensten RA
0,0701AL04,Åsgårdstrand,None,None,None,RENSEANLEGG,701,None,None,013.4,10.46635,59.35906,None,None,None,Åsgårdstrand RA


In [5]:
# The remaining unmatched RAs can be matched manually based on the output above
# Dict values below are the original names used in the Martini Excel file
manually_matched = {
    "0101AL02": "Bakke",  # Bakke (Halden)
    "0106AL00": "Øra RA",  # Øra (Fredrikstad)
    "0136AL00": "Fuglevik RA",  # Fuglevik (Rygge)
    "0602AL06": "Solumstrand RA",
    "0709AL01": "Lillevik RA",  # Lillevik (Larvik)
    "0105AL00": "Alvein RA",  # Alvim (Sarpsborg). Alvein is a typo in the Excel file
    "0214AL23": "Nordre Folle RA",  # Nordre Follo (Ås). Nordre Folle is a typo in the Excel file
    "0220AL01": "VEAS",  # VEAS (Asker)
    "0704AL40": "Tønsberg RA",  # Vallø (Tønsberg)
    "0805AL01": "Knarrdalstrand RA",  # Knarrdalsstrand (Porsgrunn). Extra 's' missing in the Excel file
}

# Get data for manually matched sites
site_list = list(manually_matched.keys())
placeholders = ",".join(f":value{i}" for i in range(len(site_list)))
params = {f"value{i}": value for i, value in enumerate(site_list)}
sql = text(
    "SELECT * FROM resa2.rid_punktkilder " f"WHERE anlegg_nr IN ({placeholders})"
)
man_match_df = pd.read_sql(sql, eng, params=params)
man_match_df["martini_name"] = man_match_df["anlegg_nr"].map(manually_matched)

# Combine into single dataset 
match_df = pd.concat([match_df, man_match_df], axis="rows").reset_index(drop=True)
assert len(match_df) == len(mar_df)

match_df.head()

,anlegg_nr,anlegg_navn,etat,opprettet,nedlagt,type,kno,adresse,postnr,regine,lon_utl,lat_utl,lon_anl,lat_anl,status,martini_name
0,0101AL07,Remmendalen,None,None,None,RENSEANLEGG,101,None,None,001.31Z,11.36011,59.12080,None,None,None,Remmendalen RA
1,0135AL01,Hestevold renseanlegg,None,None,None,RENSEANLEGG,135,None,None,003.120,10.79192,59.31496,None,None,None,Hestevold RA
2,0104AL01,Kambo,None,None,None,RENSEANLEGG,104,None,None,003.20,10.68650,59.47445,None,None,None,Kambo RA
3,0215AL35,Frogn,None,None,None,RENSEANLEGG,215,None,None,004.3,10.64256,59.64207,None,None,None,Frogn RA
4,0301AL01,Bekkelaget,None,None,None,RENSEANLEGG,301,None,None,006.21,10.76701,59.88296,None,None,None,Bekkelaget RA


## 2. Get data for Martini wastewater sites

Phil's spreadsheet includes data for 2018. As a check, get the 2018 data for these sites from the TEOTIL2 database for comparison.

In [6]:
def get_annual_renseanlegg_data(year, engine, site_list, par_list=["Tot-N", "Tot-P"]):
    """Get annual renseanlegg data from RESA2.

    Args:
        year:      Int. Year of interest
        engine:    SQL-Alchemy 'engine' object already connected
                   to RESA2
        site_list: List. Anlegg codes of interest
        par_list:  List. Parameters defined in
                   RESA2.RID_PUNKTKILDER_OUTPAR_DEF

    Returns:
        Dataframe
    """

    sql = text(
        "SELECT anlegg_nr, anlegg_navn, regine, name, SUM(value) AS value FROM ( "
        "  SELECT b.anlegg_nr, "
        "         b.anlegg_navn, "
        "         b.regine, "
        "         b.type, "
        "         c.name, "
        "         (a.value*d.factor) AS value "
        "  FROM resa2.rid_punktkilder_inpar_values a, "
        "  resa2.rid_punktkilder b, "
        "  resa2.rid_punktkilder_outpar_def c, "
        "  resa2.rid_punktkilder_inp_outp d "
        "  WHERE a.anlegg_nr = b.anlegg_nr "
        "  AND a.inp_par_id = d.in_pid "
        "  AND c.out_pid = d.out_pid "
        "  AND year = %s) "
        "WHERE type = 'RENSEANLEGG' "
        "GROUP BY anlegg_nr, anlegg_navn, regine, type, name" % year
    )

    ren_df = pd.read_sql(sql, engine)
    ren_df = ren_df.query("anlegg_nr in @site_list")

    # Only continue if data
    if len(ren_df) == 0:
        print("    No renseanlegg data for %s." % year)

        return None

    else:
        # Pivot
        ren_df = ren_df.pivot(
            index=["anlegg_nr", "anlegg_navn", "regine"], columns="name", values="value"
        ).copy()

        # If no data for pars, add cols of 0
        for par in par_list:
            if par not in ren_df.columns:
                print(f"    No renseanlegg data for {par} in {year}.")
                ren_df[par] = 0

        # Tidy
        ren_df = ren_df[par_list]
        cols = ["ren_%s_tonnes" % i.lower() for i in ren_df.columns]
        ren_df.columns = cols
        ren_df.columns.name = ""
        ren_df.reset_index(inplace=True)
        ren_df.dropna(
            subset=[
                "regine",
            ],
            inplace=True,
        )
        ren_df.dropna(subset=cols, how="all", inplace=True)

        return ren_df

In [7]:
# Get data for 2018
site_list = match_df["anlegg_nr"].tolist()
ren_df = get_annual_renseanlegg_data(2018, eng, site_list, par_list=["Tot-N", "Tot-P"])

# Join to Phil's data to compare values
comp_df = pd.merge(
    ren_df, match_df[["anlegg_nr", "martini_name"]], how="left", on="anlegg_nr"
)
comp_df = pd.merge(
    comp_df,
    mar_df[["Navn", "TN_ton_yr", "TP_ton_yr"]],
    how="left",
    left_on="martini_name",
    right_on="Navn",
)
comp_df

,anlegg_nr,anlegg_navn,regine,ren_tot-n_tonnes,ren_tot-p_tonnes,martini_name,Navn,TN_ton_yr,TP_ton_yr
0,0101AL02,Bakke,001.2220,1.481700,0.391005,Bakke,Bakke,39.100,0.391
1,0101AL07,Remmendalen,001.31Z,114.130100,0.970275,Remmendalen RA,Remmendalen RA,114.130,0.970
2,0104AL01,Kambo,003.20,84.433090,0.637795,Kambo RA,Kambo RA,84.400,0.640
3,0105AL00,Alvim Renseanlegg,002.A4,243.046680,1.928580,Alvein RA,Alvein RA,243.000,1.930
4,0106AL00,Renseanlegg Øra,002.12,476.934010,4.943080,Øra RA,Øra RA,476.900,4.943
5,0135AL01,Hestevold renseanlegg,003.120,21.862990,0.221275,Hestevold RA,Hestevold RA,21.900,0.220
6,0136AL00,Fuglevik renseanlegg,003.17Z,180.909980,2.035420,Fuglevik RA,Fuglevik RA,180.900,2.030
7,0214AL23,Nordre Follo RA,005.4B,56.020015,1.186585,Nordre Folle RA,Nordre Folle RA,56.000,1.190
8,0215AL35,Frogn,004.3,56.058600,0.291175,Frogn RA,Frogn RA,56.100,0.290
9,0220AL01,Sentralrenseanlegg Ve,009.21,936.438885,28.559280,VEAS,VEAS,936.000,28.600


In the table above, `ren_tot-n_tonnes` and `ren_tot-p_tonnes` are the values from the TEOTIL2 database and `TN_ton_yr` and `TP_ton_yr` are taken from Phil's spreadsheet. Overall this looks good, except:

 * I think there's an error in Phil's Excel sheet for Bakke: `TN_ton_yr` should be 1.5 (or [perhaps 0.84](https://www.norskeutslipp.no/no/Diverse/Virksomhet/?CompanyID=11079)?), not 39.1.
 
 * Values for TOTN at Åsgårdstrand are a bit different. Phil's figures agree with the values on [Norske Utslipp](https://www.norskeutslipp.no/no/Diverse/Virksomhet/?CompanyID=9552), whereas the TEOTIL figures match the values supplied by SSB.
 
 * Values for TOTN at Rakkestad are a bit different. Again, Phil's figures agree with the values on [Norske Utslipp](https://www.norskeutslipp.no/no/Diverse/Virksomhet/?CompanyID=9466), whereas the TEOTIL figures match the values supplied by SSB.
 
For Åsgårdstrand and Rakkestad the differences are small. I suspect the discrepancy is due to these sites not reporting their 2018 data in time for SSB's annual reporting. When this happens, SSB estimates fluxes based on the number of people connected to each site. They then publish their annual statistics and send me their data for modelling with TEOTIL. If these sites submitted measured data after the deadline, the values would be eventually updated on Norske Utslipp, but not reflected in SSB's "official" statistics for 2018. Overall, I don't think it matters much, but **the difference at Bakke is larger and should probably be checked**.

## 3. Update model input files

The code in this section reads the original TEOTIL2 input files from GitHub, subtracts TOTN and TOTP discharges for the sites already in the Martini model, and then saves a set of modified input files for use later.

In [8]:
st_yr, end_yr = 2015, 2019

for year in range(st_yr, end_yr + 1):
    # Read original input file
    url = f"https://raw.githubusercontent.com/NIVANorge/teotil2/main/data/norway_annual_input_data/input_data_{year}.csv"
    df = pd.read_csv(url)

    # Get inputs for sites already in Martini model
    ren_df = get_annual_renseanlegg_data(
        year, eng, site_list, par_list=["Tot-N", "Tot-P"]
    )
    ren_df = ren_df.groupby("regine").sum(numeric_only=True).reset_index()
    ren_df.rename(
        columns={
            "ren_tot-n_tonnes": "martini_tot-n_tonnes",
            "ren_tot-p_tonnes": "martini_tot-p_tonnes",
        },
        inplace=True,
    )

    # Join
    df = pd.merge(df, ren_df, how="left", on="regine")

    # Subtract discharges for sites already in Martini
    for par in ["tot-n", "tot-p"]:
        df[f"martini_{par}_tonnes"].fillna(0, inplace=True)
        df[f"ren_{par}_tonnes"] = df[f"ren_{par}_tonnes"] - df[f"martini_{par}_tonnes"]
        df[f"all_point_{par}_tonnes"] = (
            df[f"all_point_{par}_tonnes"] - df[f"martini_{par}_tonnes"]
        )
        df[f"all_sources_{par}_tonnes"] = (
            df[f"all_sources_{par}_tonnes"] - df[f"martini_{par}_tonnes"]
        )
        del df[f"martini_{par}_tonnes"]

    # Save modified input file
    csv_path = f"../data/teotil2_input_minus_martini_renseanlegg/input_data_minus_martini_ra_{year}.csv"
    df.to_csv(csv_path, index=False)

## 4. Run model

Run TEOTIL2 with the modified input files.

In [9]:
# Folder containing core data
data_fold = r"/home/jovyan/shared/common/JES/teotil2/data/core_input_data"

# Folder for annual model input files
input_fold = r"../data/teotil2_input_minus_martini_renseanlegg"

# Folder for annual model output files
output_fold = r"../data/teotil2_output_minus_martini_renseanlegg"

In [10]:
# Loop over years
for year in range(st_yr, end_yr + 1):
    print("Processing:", year)

    # Run model
    csv_path = os.path.join(input_fold, f"input_data_minus_martini_ra_{year}.csv")
    g = teo.model.run_model(csv_path)

    # Save results
    out_csv = os.path.join(output_fold, f"teotil2_results_minus_martini_ra_{year}.csv")
    df = teo.model.model_to_dataframe(g, out_path=out_csv)

Processing: 2015


/opt/conda/lib/python3.10/site-packages/teotil2/model.py:67: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated in favour of `both` or `neither`.
  df[f"trans_{par}"].between(0, 1, inclusive=True).all()


Processing: 2016


/opt/conda/lib/python3.10/site-packages/teotil2/model.py:67: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated in favour of `both` or `neither`.
  df[f"trans_{par}"].between(0, 1, inclusive=True).all()


Processing: 2017


/opt/conda/lib/python3.10/site-packages/teotil2/model.py:67: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated in favour of `both` or `neither`.
  df[f"trans_{par}"].between(0, 1, inclusive=True).all()


Processing: 2018


/opt/conda/lib/python3.10/site-packages/teotil2/model.py:67: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated in favour of `both` or `neither`.
  df[f"trans_{par}"].between(0, 1, inclusive=True).all()


Processing: 2019


/opt/conda/lib/python3.10/site-packages/teotil2/model.py:67: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated in favour of `both` or `neither`.
  df[f"trans_{par}"].between(0, 1, inclusive=True).all()
